In [1]:
#Importing libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import apimoex

from data_loader import index_history
from data_loader import ticker_prices, get_ticker_history
from pypfopt import expected_returns, risk_models, EfficientFrontier, objective_functions
from optimizer import optimizer_for_tickers

import warnings
warnings.filterwarnings('ignore')

In [16]:
df = pd.read_csv(
    r"C:\Users\sofya\Desktop\Export_ru_securities-list_20250405.csv",
    encoding='utf-8',  # кодировка Windows, часто используется в России
    sep=';',            # разделитель — точка с запятой
    on_bad_lines='skip' # игнорируем строки с ошибками
)
df_stocks = df[df['SUPERTYPE'] == 'Депозитарные расписки']
tickers = df_stocks['TRADE_CODE']
tickers_list = tickers.tolist()
tickers_list

['ETLN', 'FIXP', 'QIWI', 'OKEY', 'OZON', 'AGRO']

In [15]:
import os
import pandas as pd
import requests
import sqlite3
from time import sleep

DB_NAME = "moex_candles.db"
DATA_DIR = "csv_data"

os.makedirs(DATA_DIR, exist_ok=True)

# --- Создание таблиц SQLite ---
def create_tables():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS securities (
            ticker TEXT PRIMARY KEY,
            name TEXT,
            boardid TEXT,
            type TEXT,
            currency TEXT,
            is_active INTEGER DEFAULT 1
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS candles (
            ticker TEXT,
            tradedate TEXT,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume REAL,
            value REAL,
            boardid TEXT,
            PRIMARY KEY (ticker, tradedate)
        )
    """)

    conn.commit()
    conn.close()

# --- Получение исторических данных с MOEX ---
def fetch_candles(ticker, board='TQBR'):
    url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/{board}/securities/{ticker}/candles.json"
    params = {
        "from": "2000-01-01",
        "interval": 24,
        "iss.meta": "off",
        "iss.json": "extended",
        "candles.columns": "begin,open,high,low,close,value,volume"
    }

    all_data = []
    start = 0

    while True:
        try:
            r = requests.get(url, params={**params, "start": start}, timeout=15)
            r.raise_for_status()
            data = r.json()

            if len(data) < 2 or "candles" not in data[1]:
                break

            candles = data[1]["candles"]
            if not candles:
                break

            all_data.extend(candles)

            start += 500
            sleep(0.2)  # чтобы не получить бан

        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
            break

    df = pd.DataFrame(all_data, columns=["begin", "open", "high", "low", "close", "value", "volume"])
    df.rename(columns={"begin": "tradedate"}, inplace=True)
    df["tradedate"] = pd.to_datetime(df["tradedate"]).dt.date  # удаляем время
    df.insert(0, "ticker", ticker)
    df["boardid"] = board

    return df

# --- Сохранение в CSV и SQLite ---
def save_data(df):
    ticker = df["ticker"].iloc[0]

    before = len(df)
    df = df.drop_duplicates(subset=["ticker", "tradedate"])
    after = len(df)
    removed = before - after
    if removed > 0:
        print(f"Removed {removed} duplicate rows for {ticker}")

    # Save to CSV
    df.to_csv(os.path.join(DATA_DIR, f"{ticker}.csv"), index=False)

    # Save to SQLite
    conn = sqlite3.connect(DB_NAME)
    try:
        df.to_sql("candles", conn, if_exists="append", index=False)
    except sqlite3.IntegrityError as e:
        print(f"❌ Ошибка вставки {ticker}: {e}")
    finally:
        conn.close()

# --- Пример работы ---
if __name__ == "__main__":
    create_tables()
    tickers = tickers_list  # заменишь на свой список

    for ticker in tickers:
        df = fetch_candles(ticker)
        if not df.empty:
            save_data(df)
            print(f"Saved: {ticker} ({len(df)} rows)")
        else:
            print(f"No data for {ticker}")


Saved: CBOM (2461 rows)
Saved: ALRS (3144 rows)
Saved: VTBR (3040 rows)
Saved: MDMG (1085 rows)
Saved: GEMC (899 rows)
Saved: VKCO (803 rows)
Saved: LENT (820 rows)
Saved: RUAL (2520 rows)
Saved: T (1344 rows)
Saved: HEAD (141 rows)
Saved: CNRU (2 rows)
Saved: ENPG (1288 rows)
Saved: YDEX (187 rows)
Saved: BSPB (2723 rows)
Saved: AQUA (2360 rows)
Saved: AFKS (2725 rows)
Saved: AFLT (4297 rows)
Saved: VSEH (193 rows)
Saved: GAZP (2725 rows)
Saved: GMKN (3042 rows)
Saved: RAGR (35 rows)
Saved: LSRG (3550 rows)
Saved: POSI (824 rows)
Saved: RENI (860 rows)
Saved: EUTR (353 rows)
Saved: IRAO (3386 rows)
Saved: X5 (69 rows)
Saved: LEAS (268 rows)
Saved: MVID (2719 rows)
Saved: MBNK (241 rows)
Saved: MAGN (2725 rows)
Saved: MTLR (4032 rows)
Saved: MTLRP (3440 rows)
Saved: MTSS (3625 rows)
Saved: MOEX (3020 rows)
Saved: LKOH (5415 rows)
Saved: BELU (1883 rows)
Saved: NLMK (2725 rows)
Saved: PIKK (3418 rows)
Saved: PLZL (2722 rows)
Saved: RTKM (5375 rows)
Saved: RTKMP (5369 rows)
Saved: SBER (

In [17]:
import os
import pandas as pd
import requests
import sqlite3
from time import sleep

DB_NAME = "moex_candles.db"
DATA_DIR = "csv_data"

os.makedirs(DATA_DIR, exist_ok=True)

# --- Создание таблиц SQLite ---
def create_tables():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS securities (
            ticker TEXT PRIMARY KEY,
            name TEXT,
            boardid TEXT,
            type TEXT,
            currency TEXT,
            is_active INTEGER DEFAULT 1
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS candles (
            ticker TEXT,
            tradedate TEXT,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume REAL,
            value REAL,
            boardid TEXT,
            PRIMARY KEY (ticker, tradedate)
        )
    """)

    conn.commit()
    conn.close()

# --- Получение исторических данных с MOEX ---
def fetch_candles(ticker, board='TQBR'):
    url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/{board}/securities/{ticker}/candles.json"
    params = {
        "from": "2000-01-01",
        "interval": 24,
        "iss.meta": "off",
        "iss.json": "extended",
        "candles.columns": "begin,open,high,low,close,value,volume"
    }

    all_data = []
    start = 0

    while True:
        try:
            r = requests.get(url, params={**params, "start": start}, timeout=15)
            r.raise_for_status()
            data = r.json()

            if len(data) < 2 or "candles" not in data[1]:
                break

            candles = data[1]["candles"]
            if not candles:
                break

            all_data.extend(candles)

            start += 500
            sleep(0.2)  # чтобы не получить бан

        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
            break

    df = pd.DataFrame(all_data, columns=["begin", "open", "high", "low", "close", "value", "volume"])
    df.rename(columns={"begin": "tradedate"}, inplace=True)
    df["tradedate"] = pd.to_datetime(df["tradedate"]).dt.date  # удаляем время
    df.insert(0, "ticker", ticker)
    df["boardid"] = board

    return df

# --- Сохранение в CSV и SQLite ---
def save_data(df):
    ticker = df["ticker"].iloc[0]

    before = len(df)
    df = df.drop_duplicates(subset=["ticker", "tradedate"])
    after = len(df)
    removed = before - after
    if removed > 0:
        print(f"Removed {removed} duplicate rows for {ticker}")

    # Save to CSV
    df.to_csv(os.path.join(DATA_DIR, f"{ticker}.csv"), index=False)

    # Save to SQLite
    conn = sqlite3.connect(DB_NAME)
    try:
        df.to_sql("candles", conn, if_exists="append", index=False)
    except sqlite3.IntegrityError as e:
        print(f"❌ Ошибка вставки {ticker}: {e}")
    finally:
        conn.close()

# --- Пример работы ---
if __name__ == "__main__":
    create_tables()
    tickers = tickers_list  # заменишь на свой список

    for ticker in tickers:
        df = fetch_candles(ticker)
        if not df.empty:
            save_data(df)
            print(f"Saved: {ticker} ({len(df)} rows)")
        else:
            print(f"No data for {ticker}")


Saved: ETLN (1290 rows)
Saved: FIXP (1016 rows)
Saved: QIWI (2710 rows)
Saved: OKEY (1073 rows)
Saved: OZON (1088 rows)
Saved: AGRO (2475 rows)


In [20]:
# --- Сохранение в CSV и SQLite ---
def save_data(df):
    ticker = df["ticker"].iloc[0]

    before = len(df)
    df = df.drop_duplicates(subset=["ticker", "tradedate"])
    after = len(df)
    removed = before - after
    if removed > 0:
        print(f"Removed {removed} duplicate rows for {ticker}")

    # Преобразуем tradedate в строку, чтобы избежать ошибок в SQLite
    df["tradedate"] = pd.to_datetime(df["tradedate"]).dt.strftime('%Y-%m-%d')

    # Append to CSV (if exists)
    csv_path = os.path.join(DATA_DIR, f"{ticker}.csv")
    if os.path.exists(csv_path):
        old_df = pd.read_csv(csv_path, parse_dates=["tradedate"])
        df = pd.concat([old_df, df], ignore_index=True)
        df.drop_duplicates(subset=["ticker", "tradedate"], inplace=True)

    df.to_csv(csv_path, index=False)

    # Преобразуем все данные в строковый формат, чтобы избежать ошибок с типами данных
    df = df.applymap(str)

    # Save to SQLite
    conn = sqlite3.connect(DB_NAME)
    try:
        df.to_sql("candles", conn, if_exists="append", index=False)
    except sqlite3.IntegrityError as e:
        print(f"❌ Ошибка вставки {ticker}: {e}")
    finally:
        conn.close()



# --- Сбор данных по старому тикеру и сохранение под новым ---
def migrate_ticker(old_ticker, new_ticker):
    df = fetch_candles(old_ticker)
    if df.empty:
        print(f"No data for {old_ticker}")
        return
    df["ticker"] = new_ticker  # переименовываем тикер
    save_data(df)
    print(f"Migrated {old_ticker} → {new_ticker}: {len(df)} rows")

# --- Пример работы ---
if __name__ == "__main__":

   
    # Миграция старых тикеров
    migration_map = {
        "FIVE": "X5",
        "MAIL": "VKCO",
        "CIAN": "CNRU",
        "HHRU": "HEAD"
    }

    for old, new in migration_map.items():
        migrate_ticker(old, new)

Migrated FIVE → X5: 1540 rows
Migrated MAIL → VKCO: 371 rows
Migrated CIAN → CNRU: 803 rows
Migrated HHRU → HEAD: 962 rows
